In [5]:
# Imports
import bs4
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from tqdm import tqdm

In [6]:
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [7]:
url = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
page = requests.get(url,timeout = 5,headers=headers)
print(page)
soup = BeautifulSoup(page.text)

<Response [200]>


f:\Anaconda\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [8]:
title = []
authors = []
subject = []
for link in soup.find_all('dd'):
    for layer in link.find('div'):
        if isinstance(layer,bs4.element.Tag):
            temp = layer.text.strip()
            if temp[0] == 'T':
                title.append(temp.split('Title:')[-1])
            elif temp[0] == 'A':
                authors.append(temp.split('Authors:\n')[-1])
            elif temp[0] == 'S':
                subject.append(temp.split('Subjects:')[-1])
                
print(f" length of title , authors and subject are {len(title)} , {len(authors)} and {len(subject)}. " )


 length of title , authors and subject are 100 , 100 and 100. 


In [13]:
abstract_list = []
for link in tqdm(soup.find_all('dt')):
    for layer in link.find('span'):
        if len(layer.string) > 5:
            series = layer.string.split("arXiv:")[-1]
            url = "https://arxiv.org/abs/" + series
            sleep(0.5)
            page = requests.get(url,timeout = 10, headers=headers)
            #print(page.status_code)
            soup_spec = BeautifulSoup(page.text)
            abstract_raw = soup_spec.find('blockquote').text.replace("\n","")
            abstract = abstract_raw.split("Abstract:")[-1]
            abstract_list.append(abstract)
            #print(abstract)
            


  0%|          | 0/100 [00:00<?, ?it/s]f:\Anaconda\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 100/100 [01:05<00:00,  1.52it/s]


In [14]:
d = {'Title': title, 'Author' : authors , 'Subject':subject, 'Abstract':abstract_list}
df_final = pd.DataFrame(data=d)
df_final.to_csv("./papers.csv",index=False)


In [16]:
df_1 = pd.read_csv("./papers.csv")
df_1.head(5)

,Title,Author,Subject,Abstract
0,Point Could Mamba: Point Cloud Learning via S...,"Tao Zhang, \nXiangtai Li, \nHaobo Yuan, \nShun...",Computer Vision and Pattern Recognition (cs.CV),"In this work, for the first time, we demonstra..."
1,Mitigating Reversal Curse via Semantic-aware ...,"Qingyan Guo, \nRui Wang, \nJunliang Guo, \nXu ...",Computation and Language (cs.CL); Artificial ...,While large language models (LLMs) have achiev...
2,An Experimental Study of Low-Latency Video St...,"Imran Khan, \nTuyen X. Tran, \nMatti Hiltunen,...",Multimedia (cs.MM); Performance (cs.PF),Low-latency video streaming over 5G has become...
3,AtP*: An efficient and scalable method for lo...,"János Kramár, \nTom Lieberum, \nRohin Shah, \n...",Machine Learning (cs.LG); Computation and Lan...,Activation Patching is a method of directly co...
4,Neural Acceleration of Incomplete Cholesky Pr...,"Joshua Dennis Booth, \nHongyang Sun, \nTrevor ...","Distributed, Parallel, and Cluster Computing ...",The solution of a sparse system of linear equa...
